## 创建问答数据集

In [1]:
import torch
from transformers import BertForQuestionAnswering
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
train_df = pd.read_csv(r'疫情政务问答助手数据集\NCPPolicies_train_20200301.csv',sep = '\t',encoding = 'utf-8')

context_df = pd.read_csv(r'疫情政务问答助手数据集\NCPPolicies_context_20200301.csv',sep = '\t',error_bad_lines=False,encoding = 'utf-8')

b'Skipping line 134: expected 2 fields, saw 3\nSkipping line 2130: expected 2 fields, saw 3\nSkipping line 2877: expected 2 fields, saw 3\nSkipping line 2955: expected 2 fields, saw 3\nSkipping line 2974: expected 2 fields, saw 3\nSkipping line 3038: expected 2 fields, saw 3\nSkipping line 3052: expected 2 fields, saw 3\nSkipping line 3053: expected 2 fields, saw 3\nSkipping line 3080: expected 2 fields, saw 3\nSkipping line 3086: expected 2 fields, saw 3\nSkipping line 3094: expected 2 fields, saw 3\nSkipping line 3115: expected 2 fields, saw 22\nSkipping line 3180: expected 2 fields, saw 3\nSkipping line 3189: expected 2 fields, saw 3\nSkipping line 3191: expected 2 fields, saw 3\nSkipping line 3197: expected 2 fields, saw 3\nSkipping line 3203: expected 2 fields, saw 3\nSkipping line 3209: expected 2 fields, saw 3\nSkipping line 3215: expected 2 fields, saw 3\nSkipping line 3611: expected 2 fields, saw 3\nSkipping line 3613: expected 2 fields, saw 3\nSkipping line 3716: expected 2 f

In [3]:
merge_df = pd.merge(context_df,train_df, how='left', on='docid')
#merge_df['text'] = merge_df['text'].apply(lambda x : str(x).replace(' ', '').replace('\t', ''))
#merge_df['question'] = merge_df['question'].apply(lambda x : str(x).replace(' ', '').replace('\t', ''))
merge_df.drop_duplicates(inplace = True)
merge_df.dropna(inplace = True)


In [4]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4997 entries, 0 to 11143
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   docid     4997 non-null   object
 1   text      4997 non-null   object
 2   id        4997 non-null   object
 3   question  4997 non-null   object
 4   answer    4997 non-null   object
dtypes: object(5)
memory usage: 234.2+ KB


In [5]:
merge_df.head()

,docid,text,id,question,answer
0,edd1413c78e534afb136f36fdc9c9a00,福建：6部门联合出台暖企措施支持复工稳岗 为解决企业复产的用工困难，经省政府同意，省人社厅、...,fdc51a7baeff3fafbae6736422783528,福建联合出台暖企措施支持复工稳岗的部门都有谁？,省人社厅、省工信厅、省教育厅、省财政厅、省交通运输厅、省卫健委
1,edd1413c78e534afb136f36fdc9c9a00,福建：6部门联合出台暖企措施支持复工稳岗 为解决企业复产的用工困难，经省政府同意，省人社厅、...,6b7b0c209ac939afa43f030d67433178,福建政府针对引入本地劳动力的、未经有关机构确认的疫情物资生产企业做何补助？,一次性用工服务奖补标准最高提到每人2000元。
3,8bab952dfa1a367da1b8e7ad864d766b,吉林多措并举保民生 2月18日，省政府新闻办围绕就业和医疗保障等民生问题召开新闻发布会。省...,2c3b39b675493830b4d1f2f4f2a436f9,吉林省各级医疗保障部门为百姓提供哪些便捷服务？,全面实行“非必须、不窗口”经办服务，通过“网上办”“掌上办”“延长时限办”“后期补办”等方式...
4,8bab952dfa1a367da1b8e7ad864d766b,吉林多措并举保民生 2月18日，省政府新闻办围绕就业和医疗保障等民生问题召开新闻发布会。省...,9a9acefa0f533ca99fd174aa89b7f600,吉林省提前预付医保基金总额达到了多少？,1.22亿元
8,a84ad5c0fc7d3755926ec5bf2b2d9dae,重庆出台援企稳岗返还政策 2月16日，记者从市人力社保局获悉，为切实减轻中小企业负担，充分发...,a505ce9785333655aae92fabba500f70,重庆援企稳岗返还政策的申请条件是什么？,依法参加社会保险并足额缴纳2019年度社会保险费（截至2019年12月31日无欠费），企业2...


In [6]:
# HERE I find the indices (start) of the selected_text from Text column and create a new column called answer_index
def answer_index(x):
    return x.text.find(str(x.answer))

merge_df["answer_index"]=merge_df.apply(answer_index,axis=1)
#merge_df["answer"] = merge_df[merge_df['answer_index'] == -1]

In [7]:
merge_df[:1].text

0    福建：6部门联合出台暖企措施支持复工稳岗 为解决企业复产的用工困难，经省政府同意，省人社厅、...
Name: text, dtype: object

In [8]:
merge_df.head()

,docid,text,id,question,answer,answer_index
0,edd1413c78e534afb136f36fdc9c9a00,福建：6部门联合出台暖企措施支持复工稳岗 为解决企业复产的用工困难，经省政府同意，省人社厅、...,fdc51a7baeff3fafbae6736422783528,福建联合出台暖企措施支持复工稳岗的部门都有谁？,省人社厅、省工信厅、省教育厅、省财政厅、省交通运输厅、省卫健委,41
1,edd1413c78e534afb136f36fdc9c9a00,福建：6部门联合出台暖企措施支持复工稳岗 为解决企业复产的用工困难，经省政府同意，省人社厅、...,6b7b0c209ac939afa43f030d67433178,福建政府针对引入本地劳动力的、未经有关机构确认的疫情物资生产企业做何补助？,一次性用工服务奖补标准最高提到每人2000元。,336
3,8bab952dfa1a367da1b8e7ad864d766b,吉林多措并举保民生 2月18日，省政府新闻办围绕就业和医疗保障等民生问题召开新闻发布会。省...,2c3b39b675493830b4d1f2f4f2a436f9,吉林省各级医疗保障部门为百姓提供哪些便捷服务？,全面实行“非必须、不窗口”经办服务，通过“网上办”“掌上办”“延长时限办”“后期补办”等方式...,1792
4,8bab952dfa1a367da1b8e7ad864d766b,吉林多措并举保民生 2月18日，省政府新闻办围绕就业和医疗保障等民生问题召开新闻发布会。省...,9a9acefa0f533ca99fd174aa89b7f600,吉林省提前预付医保基金总额达到了多少？,1.22亿元,1466
8,a84ad5c0fc7d3755926ec5bf2b2d9dae,重庆出台援企稳岗返还政策 2月16日，记者从市人力社保局获悉，为切实减轻中小企业负担，充分发...,a505ce9785333655aae92fabba500f70,重庆援企稳岗返还政策的申请条件是什么？,依法参加社会保险并足额缴纳2019年度社会保险费（截至2019年12月31日无欠费），企业2...,304


In [9]:
train_data = []

def make_train_data(df):      
    for ind in df.index:
        main_d=dict()
        context=df.loc[ind,"text"].lower()
        main_d["context"] = context
        qas=[]
        inside_qas={}
        inside_qas["id"]=df.loc[ind,"docid"]
        inside_qas["question"]=df.loc[ind,"question"]
        inside_answer=[{"text":df.loc[ind,"answer"].lower(),"answer_start": int(df.loc[ind,"answer_index"])}]
        inside_qas["answers"]=inside_answer
        inside_qas["is_impossible"]=False
        qas.append(inside_qas)
        main_d["qas"]=qas
        train_data.append({"title": " ","paragraphs": [main_d]})
    return train_data
          

In [10]:
def answer_index(x):
    return x.text.find(str(x.answer))

merge_df["answer_index"]=merge_df.apply(answer_index,axis=1)

In [11]:
train_data = make_train_data(merge_df)


In [12]:
train_data[0]

{'title': ' ',
 'paragraphs': [{'context': '福建：6部门联合出台暖企措施支持复工稳岗 为解决企业复产的用工困难，经省政府同意，省人社厅、省工信厅、省教育厅、省财政厅、省交通运输厅、省卫健委联合下发通知，出台一系列暖企措施支持疫情防控期间复工稳岗。 通知明确，切实发挥各级农民工工作领导小组办公室的统筹协调作用, 加强劳务用工有效对接，对具备外出务工条件、可成规模输送到我省用工地，并在出行前14天内及在途没有相关症状的，可由用工地和输出地联合开展“点对点、一站式”直达企业的专门运输。省级公共就业服务机构可与主要劳务输出省份签订劳务协作协议、设立劳务协作工作站，对每个工作站给予一次性10万元就业服务经费补助。鼓励优先聘用本地劳务人员。 未经省应对新冠肺炎疫情工作有关机构确认的疫情防控急需物资生产企业引进劳动力的，一次性用工服务奖补标准最高提到每人2000元。对上述企业坚持在生产一线工作的职工，给予每人每天100元的生活补助，纳入一次性用工服务奖补范畴。对春节当月至疫情一级响应结束月，采取稳定职工队伍保持连续生产的企业，给予一次性稳就业奖补。 加大失业保险稳岗返还力度，将中小微企业稳岗返还政策裁员率标准调整为不高于上年度全国调查失业率的控制目标，对参保职工30人（含）以下的企业，裁员率调整为不超过企业参保职工总数的20%。对不裁员或少裁员，符合条件的参保企业，可返还其上年度实际缴纳失业保险费的50%。对受疫情影响面临暂时性生产经营困难且恢复有望、坚持不裁员或少裁员、符合条件的参保企业，按6个月的当地月人均失业保险金和参保职工人数落实失业保险稳岗返还政策。 加强职业技能培训，鼓励技工院校学生在符合疫情防控条件下参加实习实训，探索简易岗前技能培训。对企业因生产急需新录用的人员，按每人200元标准一次性给予企业简易岗前技能培训补贴。鼓励实施线上培训，对受疫情影响的企业，在停工期、恢复期组织职工参加各类线上或线下职业培训的，可按规定纳入补贴类培训范围。 通知要求，各地要着力提升政策措施的精准度和有效性，提升各类企业享受政策措施的获得感。各类企业要落实落细防控主体责任，严格落实返岗信息登记、班车错峰接送、员工分散用餐、体温监测等具体应对措施，确保复工稳岗和疫情防控两不误。（记者 潘园园） ',
   'qas': [{'id': 'edd1

In [13]:
store_data = {'data':train_data}

In [14]:
import json

with open(r'Coronavirus2019.json', 'w',encoding = 'utf-8') as outfile:
    json.dump(store_data, outfile,ensure_ascii = False,indent=4)


In [94]:
import logging,json

def generate_examples(filepath):
        ids,contexts,questions,answers = [], [], [], []
        logging.info("generating examples from = %s", filepath)
        with open(filepath, encoding="utf-8") as f:
            squad = json.load(f)
            for article in squad["data"]:
                title = article.get("title", "").strip()
                for paragraph in article["paragraphs"]:
                    
                    for qa in paragraph["qas"]:
                        contexts.append(paragraph["context"].strip())
                        questions.append(qa["question"].strip())
                        ids.append (qa["id"])
                        if qa["answers"]:
                            answers.append([answer["text"].strip() for answer in qa["answers"]][0])
                        else:
                            answers.append(-1)

        return ids,contexts,questions,answers

In [147]:
ids,contexts,questions,answers = generate_examples(r'C:\Users\Administrator\Desktop\2020.07.21基于transformers的智能问答\DRCD-master\DRCD_dev.json')

In [148]:
len(ids),len(contexts),len(questions),len(answers)

(3524, 3524, 3524, 3524)

In [149]:
ids[1],contexts[1],questions[1],answers[1]

('1147-5-2',
 '在歐洲，梵語的學術研究，由德國學者陸特和漢斯雷頓開創。後來威廉·瓊斯發現印歐語系，也要歸功於對梵語的研究。此外，梵語研究，也對西方文字學及歷史語言學的發展，貢獻不少。1786年2月2日，亞洲協會在加爾各答舉行。會中，威廉·瓊斯發表了下面這段著名的言論：「梵語儘管非常古老，構造卻精妙絕倫：比希臘語還完美，比拉丁語還豐富，精緻之處同時勝過此兩者，但在動詞詞根和語法形式上，又跟此兩者無比相似，不可能是巧合的結果。這三種語言太相似了，使任何同時稽考三者的語文學家都不得不相信三者同出一源，出自一種可能已經消逝的語言。基於相似的原因，儘管缺少同樣有力的證據，我們可以推想哥德語和凱爾特語，雖然混入了迥然不同的語彙，也與梵語有著相同的起源；而古波斯語可能也是這一語系的子裔。」',
 '印歐語系因為哪一門語言而被發現？',
 '梵語')

In [150]:
train_df = pd.DataFrame({'docid':ids,'context':	contexts, 'question':questions,'answer':answers})

In [151]:
train_df.head()

,docid,context,question,answer
0,1147-5-1,在歐洲，梵語的學術研究，由德國學者陸特和漢斯雷頓開創。後來威廉·瓊斯發現印歐語系，也要歸功於...,陸特和漢斯雷頓開創了哪一地區對梵語的學術研究？,歐洲
1,1147-5-2,在歐洲，梵語的學術研究，由德國學者陸特和漢斯雷頓開創。後來威廉·瓊斯發現印歐語系，也要歸功於...,印歐語系因為哪一門語言而被發現？,梵語
2,1147-5-3,在歐洲，梵語的學術研究，由德國學者陸特和漢斯雷頓開創。後來威廉·瓊斯發現印歐語系，也要歸功於...,誰認為希臘語、拉丁語與梵語是出自於同一門可能已經消失的語言？,威廉·瓊斯
3,1147-6-1,梵語在社交中口頭使用，並且在早期古典梵語文獻的發展中維持口頭傳統。在印度，書寫形式是當梵語發...,印度的梵語的書寫形式是到何時才出現的？,當梵語發展成俗語之後
4,1147-6-2,梵語在社交中口頭使用，並且在早期古典梵語文獻的發展中維持口頭傳統。在印度，書寫形式是當梵語發...,天城文在何時成為梵語的標準書寫系統？,19世紀晚期


In [153]:
def answer_index(x):
    return x.context.find(str(x.answer))

train_df["answer_index"]=train_df.apply(answer_index,axis=1)

In [154]:
train_df.to_csv('aaacomm.csv')

## 合并多个json文件

In [21]:
import os,glob,json

data = []

filesdir =r'conbined_train_data'
for f in glob.glob(filesdir + "\\*.json"):
    print(f)
    with open(f, encoding = 'utf-8') as infile:
        data.append(json.load(infile))

with open("conbined_train_data\merged_results.json",'w', encoding = 'utf-8') as outfile:
    json.dump(data, outfile,ensure_ascii = False,indent=4)

conbined_train_data\NCPPolicies_train.json
conbined_train_data\train-zen-v1.0.json


### 【Squad形式】
```
{'answers': {'answer_start': [148], 'text': ['圣伯纳黛特苏比罗斯。']},
 'context': '在建筑上，这所学校是天主教的。在主楼的金色穹顶上是一座圣母玛利亚的金色雕像。紧靠着主楼的前面，面对着它，是一尊双臂高举的基督铜像，上面写着“威尼斯和我的天”。主楼旁边是圣心大教堂。紧跟在大教堂后面的是石窟，一个马里祈祷和思考的地方。这是法国卢尔德石窟的复制品，据说1858年圣母玛利亚在那里出现在圣伯纳黛特苏比罗斯。在主干道的尽头（通过3座雕像和金色穹顶直接连接），是一座简单的现代玛丽石像。',
 'id': '5733be284776f41900661182',
 'question': '据称，1858年，圣母玛利亚在法国卢尔德出现在谁的面前？',
 'title': '圣母大学'}
```

### 【预期形式】

``` 
    {
        "context": "福建：6部门联合出台暖企措施支持复工稳岗 为解决企业复产的用工困难，经省政府同意，省人社厅、省工信厅、省教育厅、省财政厅、省交通运输厅、省卫健委联合下发通知，出台一系列暖企措施支持疫情防控期间复工稳岗。 通知明确，切实发挥各级农民工工作领导小组办公室的统筹协调作用, 加强劳务用工有效对接，对具备外出务工条件、可成规模输送到我省用工地，并在出行前14天内及在途没有相关症状的，可由用工地和输出地联合开展“点对点、一站式”直达企业的专门运输。省级公共就业服务机构可与主要劳务输出省份签订劳务协作协议、设立劳务协作工作站，对每个工作站给予一次性10万元就业服务经费补助。鼓励优先聘用本地劳务人员。 未经省应对新冠肺炎疫情工作有关机构确认的疫情防控急需物资生产企业引进劳动力的，一次性用工服务奖补标准最高提到每人2000元。对上述企业坚持在生产一线工作的职工，给予每人每天100元的生活补助，纳入一次性用工服务奖补范畴。对春节当月至疫情一级响应结束月，采取稳定职工队伍保持连续生产的企业，给予一次性稳就业奖补。 加大失业保险稳岗返还力度，将中小微企业稳岗返还政策裁员率标准调整为不高于上年度全国调查失业率的控制目标，对参保职工30人（含）以下的企业，裁员率调整为不超过企业参保职工总数的20%。对不裁员或少裁员，符合条件的参保企业，可返还其上年度实际缴纳失业保险费的50%。对受疫情影响面临暂时性生产经营困难且恢复有望、坚持不裁员或少裁员、符合条件的参保企业，按6个月的当地月人均失业保险金和参保职工人数落实失业保险稳岗返还政策。 加强职业技能培训，鼓励技工院校学生在符合疫情防控条件下参加实习实训，探索简易岗前技能培训。对企业因生产急需新录用的人员，按每人200元标准一次性给予企业简易岗前技能培训补贴。鼓励实施线上培训，对受疫情影响的企业，在停工期、恢复期组织职工参加各类线上或线下职业培训的，可按规定纳入补贴类培训范围。 通知要求，各地要着力提升政策措施的精准度和有效性，提升各类企业享受政策措施的获得感。各类企业要落实落细防控主体责任，严格落实返岗信息登记、班车错峰接送、员工分散用餐、体温监测等具体应对措施，确保复工稳岗和疫情防控两不误。（记者 潘园园） ",
        "qas": [
            {
                "id": "edd1413c78e534afb136f36fdc9c9a00",
                "question": "福建联合出台暖企措施支持复工稳岗的部门都有谁？",
                "answers": [
                    {
                        "text": "省人社厅、省工信厅、省教育厅、省财政厅、省交通运输厅、省卫健委",
                        "answer_start": 41
                    }
                ],
                "is_impossible": false
            }
        ]
    }
    
```